<center>
    <img src="https://gitlab.com/ibm/skills-network/courses/placeholder101/-/raw/master/labs/module%201/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>


# **Extract Transform Load (ETL) Lab**


Estimated time needed: **30** minutes


## Objetivo

Neste Laboratório apresentamos:

*   Ler arquivos CSV e JSON.
*   Extrair dados dos tipos de arquivos acima.
*   Transformar os dados.
*   Salvar os dados transformados em um formato pronto para carregar que os Engenheiros de dados usam para carregar em RDMS.


Importar os módulos e funções


In [1]:
import glob                         # Este módulo ajuda na seleção dos arquivos
import pandas as pd                 # Este módulo ajuda no processamento de arquivos CSV
import xml.etree.ElementTree as ET  # Este módulo ajuda no processamento de arquivos XML.
from datetime import datetime

## Download dos Arquivos


In [2]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip

--2021-12-23 10:53:47--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: 'source.zip'

source.zip          100%[===================>]   2.64K  --.-KB/s    in 0s      

2021-12-23 10:53:48 (369 MB/s) - 'source.zip' saved [2707/2707]



## Descompactar os arquivos


In [3]:
!unzip source.zip

Archive:  source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  inflating: source2.json            
  inflating: source1.xml             
  inflating: source2.xml             
  inflating: source3.xml             


## Definir os Caminhos


In [4]:
tmpfile    = "temp.tmp"               # arquivo usado para guardar todos os arquivos extraídos
logfile    = "logfile.txt"            # todos os logs de eventos são gravados nesse arquivo
targetfile = "dados_transformados.csv"   # arquivo onde os dados transformardos são gravados

## Extrair


### Função para extrair CSV


In [5]:
def extrair_csv(arquivo_para_processar):
    dataframe = pd.read_csv(arquivo_para_processar)
    dataframe.rename(columns={'name': 'nome', 'height':'altura', 'weight':'peso'}, inplace = True)
    return dataframe

### Função para extrair JSON


In [6]:
def extrair_json(arquivo_para_processar):
    dataframe = pd.read_json(arquivo_para_processar,lines=True)
    dataframe.rename(columns={'name': 'nome', 'height':'altura', 'weight':'peso'}, inplace = True)
    return dataframe

### Função para extrair XML


In [7]:
def extrair_xml(arquivo_para_processar):
    dataframe = pd.DataFrame(columns=["nome", "altura", "peso"])
    tree = ET.parse(arquivo_para_processar)
    root = tree.getroot()
    for pessoa in root:
        nome = pessoa.find("name").text
        altura = float(pessoa.find("height").text)
        peso = float(pessoa.find("weight").text)
        dataframe = dataframe.append({"nome":nome, "altura":altura, "peso":peso}, ignore_index=True)
    return dataframe

### Extract Function


In [8]:
def extrair():
    dados_extraidos = pd.DataFrame(columns=['nome','altura','peso']) # cria um dataframe vazio para segurar os dados extraídos
    
    #processar todos arquivos csv
    for arquivocsv in glob.glob("*.csv"):
        dados_extraidos = dados_extraidos.append(extrair_csv(arquivocsv), ignore_index=True)
        
    #processar todos arquivos json
    for arquivojson in glob.glob("*.json"):
        dados_extraidos = dados_extraidos.append(extrair_json(arquivojson), ignore_index=True)
    
    #processar todos arquivos xml
    for arquivoxml in glob.glob("*.xml"):
        dados_extraidos = dados_extraidos.append(extrair_xml(arquivoxml), ignore_index=True)
        
    return dados_extraidos

## Transformar


A Função Transformar faz as seguintes tarefas.

1.  Converte altura que está em polegadas para milimetros
2.  Converte peso que está em libras para quilogramas


In [9]:
def transformar(data):
        #Converte altura que está em polegadas para milimetros
        #Converte o tipo de dados da coluna em real(float)
        #data.altura = data.altura.astype(float)
        #Converte polegadas em metros e arredonda para 2 casas decimais (uma polegada é 0.0254 metros)
        data['altura'] = round(data.altura * 0.0254,2)
        
        #Converte peso que está em libras para quilogramas
        #Converte o tipo do dado da coluna em real(float)
        #data.weight = data.weight.astype(float)
        #Convert pounds to kilograms and round off to two decimals(one pound is 0.45359237 kilograms)
        data['peso'] = round(data.peso * 0.45359237,2)
        return data

## Carregando


In [10]:
def carregar(targetfile,dados_para_carregar):
    dados_para_carregar.to_csv(targetfile)  

## Registrando


In [11]:
def log(mensagem):
    formato_timestamp = '%Y-%h-%d-%H:%M:%S' # Ano-NomeMes-Dia-Hora-Minuto-Segundo
    agora = datetime.now() # pegar o timestamp atual
    timestamp = agora.strftime(formato_timestamp)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + mensagem + '\n')

## Executando Processo ETL


In [12]:
log("PROCESSO ETL INICIADO")

In [13]:
log("FASE DE EXTRAÇÃO INICIADA")
dados_extraidos = extrair()
log("FASE DE EXTRAÇÃO FINALIZADA")
dados_extraidos

,nome,altura,peso
0,alex,65.78,112.99
1,ajay,71.52,136.49
2,alice,69.40,153.03
3,ravi,68.22,142.34
4,joe,67.79,144.30
5,alex,65.78,112.99
6,ajay,71.52,136.49
7,alice,69.40,153.03
8,ravi,68.22,142.34
9,joe,67.79,144.30


In [14]:
log("FASE TRANSFORMAR INICIADA")
dados_transformados = transformar(dados_extraidos)
log("FASE TRANSFORMAR FINALIZADA")
dados_transformados

,nome,altura,peso
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
5,alex,1.67,51.25
6,ajay,1.82,61.91
7,alice,1.76,69.41
8,ravi,1.73,64.56
9,joe,1.72,65.45


In [15]:
log("FASE CARREGAR INICIADA")
carregar(targetfile,dados_transformados)
log("FASE CARREGAR FINALIZADA")

In [16]:
log("TRABALHO ETL FINALIZADO")

# Exercício


Usando o exemplo acima complete os exercícios abaixo.


## Download dos Arquivos


In [17]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip

--2021-12-23 10:53:49--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/datasource.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4249 (4.1K) [application/zip]
Saving to: 'datasource.zip'

datasource.zip      100%[===================>]   4.15K  --.-KB/s    in 0s      

2021-12-23 10:53:50 (253 MB/s) - 'datasource.zip' saved [4249/4249]



## Descompactar os Arquivos


In [18]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


## Sobre os Dados


O Arquivo `dealership_data` contêm arquivos CSV, JSON, e XML de carros usados contêm recursos chamados `modelo_carro`, `ano_fabricação`, `preço`, and `combustível`.


## Definir Caminhos


In [19]:
tmpfile    = "dealership_temp.tmp"               # arquivo usado para guardar todos os dados extraídos
logfile    = "dealership_logfile.txt"            # todos logs de eventos são salvos neste arquivo
targetfile = "dealership_transformed_data.csv"   # arquivo onde os dados transformados são salvos

## Extrair


### Questão 1: Função Extrair CSV


In [20]:
def extrair_csv2(arquivo_para_processar):
    dataframe = pd.read_csv(arquivo_para_processar)
    dataframe.rename(columns={'car_model':'modelo_carro','year_of_manufacture':'ano_de_fabricacao', 'price':'preco', 'fuel':'combustivel'}, inplace = True)
    
    return dataframe

### Questão 2: Função Extrair JSON


In [21]:
def extrair_json2(arquivo_para_processar):
    dataframe = pd.read_json(arquivo_para_processar,lines=True)
    dataframe.rename(columns={'car_model':'modelo_carro','year_of_manufacture':'ano_de_fabricacao', 'price':'preco', 'fuel':'combustivel'}, inplace = True)

    return dataframe

### Questão 3: Função Extrair XML


In [22]:
def extrair_xml2(arquivo_para_processar):
    dataframe = pd.DataFrame(columns=['modelo_carro','ano_de_fabricacao', 'preco', 'combustivel'])
    tree = ET.parse(arquivo_para_processar)
    root = tree.getroot()
    for carro in root:
        modelo_carro = carro.find("car_model").text
        ano_de_fabricacao = int(carro.find("year_of_manufacture").text)
        preco = float(carro.find("price").text)
        combustivel = carro.find("fuel").text
        dataframe = dataframe.append({"modelo_carro":modelo_carro,"ano_de_fabricacao":ano_de_fabricacao,"preco":preco, "combustivel":combustivel},ignore_index=True)
    return dataframe


### Questão 4: Função Extrair


In [23]:
def extrair2():
    dados_extraidos = pd.DataFrame(columns=['modelo_carro','ano_de_fabricacao', 'preco', 'combustivel'])

    #processar todos arquivos csv
    for arquivo_csv in glob.glob("dealership_data/*.csv"):
        dados_extraidos = dados_extraidos.append(extrair_csv2(arquivo_csv), ignore_index=True)

    #processar todos arquivos json
    for arquivo_json in glob.glob("dealership_data/*.json"):
        dados_extraidos = dados_extraidos.append(extrair_json2(arquivo_json), ignore_index=True)

    #processar todos arquivos xml
    for arquivo_xml in glob.glob("dealership_data/*.xml"):
       dados_extraidos = dados_extraidos.append(extrair_xml2(arquivo_xml), ignore_index=True)

    return dados_extraidos

In [24]:
extrair2()

,modelo_carro,ano_de_fabricacao,preco,combustivel
0,ritz,2014,5000.000000,Petrol
1,sx4,2013,7089.552239,Diesel
2,ciaz,2017,10820.895522,Petrol
3,wagon r,2011,4253.731343,Petrol
4,swift,2014,6865.671642,Diesel
...,...,...,...,...
85,etios liva,2014,7089.552239,Diesel
86,innova,2017,29477.611940,Petrol
87,fortuner,2010,13805.970149,Diesel
88,corolla altis,2011,6492.537313,Petrol


## Transformar


### Questão 5: Transformar

Arredondar a coluna `preço` para 2 casas decimais


In [25]:
def transformar(data):
    data['preco'] = round(data.preco,2)
    return data

## Carregamento


### Questão 6: Carregar


In [26]:
def carregar(targetfile,dados_para_carregar):
    dados_para_carregar.to_csv(targetfile)

## Registrar


### Questão 7: Registro



In [27]:
def log(mensagem):
    timestamp_format = '%H:%M:%S-%h-%d-%Y' #Hora-Minuto-Segundo-NomedoMes-Dia-Ano
    agora = datetime.now() #pega o timestamp corrente
    timestamp = agora.strftime(timestamp_format)
    with open("dealership_logfile.txt","a") as f:
        f.write(timestamp + ',' + mensagem + '\n')

## Executando Processo ETL


### Questão 8: Processo ETL



In [28]:
log("TRABALHO ETL INICIADO")

# Registra o inicio da Extração
log("FASE EXTRAIR INICIADA")
# Chama a Função Extrair
dados_extraidos = extrair2()

# Registra da fase Extrair
log("FASE EXTRAIR FINALIZADA")

# Registra o início da Fase Tranformação
log("FASE TRANSFORMAR INICIADA")

# Chama a Função Transformar
dados_transformados = transformar(dados_extraidos)

# Registra o Fim da Fase Transformar
log("FASE TRANSFORMAR FINALIZADA")


# Registra o início da Fase Carregar
log("FASE CARREGAR INICIADA")

# Chama a Função Carregar
carregar(targetfile,dados_transformados)
# Registra Fim da Fase Carregar
log("FASE CARREGAR FINALIZADA")

# Registra o Fim do Processo ETL
log("PROCESSO ETL FINALIZADO")

## Authors


Ramesh Sannareddy

Joseph Santarcangelo

Azim Hirjani


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2021-01-01&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork-23455645&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
